# Interactive heatmap demo using `bokeh`

In [1]:
# must go first
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Reloads functions each time so you can edit a script 
# and not need to restart the kernel
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import IFrame

import warnings
warnings.filterwarnings('ignore')

Please make sure bokeh      2.2.3

Must use the code below to get Bokeh working within the notebook

In [2]:
import bokeh
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
import vishelper as vh

## Create fake data

In [4]:
df = pd.DataFrame()
df['date'] = pd.date_range(start='1/1/2020', end='12/31/2020')

Create fake values for plotting

In [5]:
df['fake_data'] = np.random.randint(0, 100, len(df))

Make a number of days having nothing recorded (`fake_data=0`)

In [6]:
df.loc[df.sample(75).index, 'fake_data'] = 0

## Create features for plotting 

Create features for day of week and week of year

In [7]:
df['dayofweek'] = df['date'].dt.dayofweek

In [8]:
df['weekofyear'] = df['date'].dt.weekofyear

Create labels

In [9]:
df["week_begins"] = df.apply(lambda x: x['date'] - pd.Timedelta("%i D" % x['dayofweek']), axis=1)
df['weekof'] = df.week_begins.dt.strftime('%B %d, %Y')

df['week_begins'] = df.week_begins.dt.strftime('%Y-%m-%d')
df['date_str'] = df.date.dt.strftime('%Y-%m-%d')

In [10]:
df.head()

,date,fake_data,dayofweek,weekofyear,week_begins,weekof,date_str
0,2020-01-01,37,2,1,2019-12-30,"December 30, 2019",2020-01-01
1,2020-01-02,76,3,1,2019-12-30,"December 30, 2019",2020-01-02
2,2020-01-03,97,4,1,2019-12-30,"December 30, 2019",2020-01-03
3,2020-01-04,0,5,1,2019-12-30,"December 30, 2019",2020-01-04
4,2020-01-05,62,6,1,2019-12-30,"December 30, 2019",2020-01-05


## Convert day of week to name of day 

In [11]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [12]:
df['dayofweek'] = df.dayofweek.apply(lambda x: days[x])

## Get range of weeks

In [13]:
weeks = df.sort_values(by='week_begins').weekof.unique()

## Set up tooltips 

Tool tips can be provided by a list of tuples. Each tuple has the label for the value to be displayed and then the name of the column containing that value, beginning with an `@`: 

```python
tooltips = [('Label of 1st piece of information', '@column_a'),
            ('Label of 2nd piece of information', '@column_b')]
```

In [14]:
tooltips = [('Date', '@date_str'), ('Fake data value', '@fake_data'), 
           ('Day of week', '@dayofweek'), ('Week of', '@weekof')]

## Create the plot

Must save at html, then can display in `IFrame`

In [15]:
save_path='fake-heatmap.html'

In [16]:
p = vh.interactive_heatmap(df,
                save_path=save_path,
                value_column='fake_data',
                y_range=days,
                ycolumn='dayofweek',
                xcolumn='weekof',
                x_range=weeks,
                colorbar_format="%d things",
                tooltips=tooltips,
                title="# of things by day of week and week fo year")

## Display the plot

In [17]:
IFrame(os.path.relpath(save_path), width=900, height=600)

## Formatting choices

Switch locations of objects: 

```python
x_axis_location="below",
toolbar_location='above',
```

In [18]:
save_pathb = 'fake-heatmap-formatted.html'

In [19]:
p = vh.interactive_heatmap(df,
                save_path=save_pathb,
                value_column='fake_data',
                y_range=days,
                ycolumn='dayofweek',
                xcolumn='weekof',
                x_range=weeks,
                colorbar_format="%d things",
                tooltips=tooltips,
                x_axis_location="below",
                toolbar_location='above',
                title="# of things by day of week and week fo year")

In [20]:
IFrame(os.path.relpath(save_pathb), width=900, height=600)

## Watermark 
For full reproducibility of results, use exact data extraction as defined at top of notebook and ensure that the environment is exactly as follows: 

In [21]:
# ! pip install watermark
%load_ext watermark
%watermark -v -m --iversions -g

matplotlib 3.1.3
pandas     1.0.3
json       2.0.9
bokeh      2.2.3
numpy      1.17.2
CPython 3.7.3
IPython 7.5.0

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 17.7.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit
Git hash   : 7b4bc94620d6cf3aad197ccf89c40e039be22978
